In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [6]:
# Import data and normalize
data = pd.read_csv("drive/My Drive/H2O_Super.csv")
X_train, X_test, y_train, y_test = train_test_split(data.values[:, 0:2], data.values[:, 2::], train_size=0.8, random_state=1)

X_mean = np.mean(X_train, 0)
X_std = np.std(X_train, 0)
y_mean = np.mean(y_train, 0)
y_std = np.std(y_train, 0)

X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std
y_train = (y_train - y_mean) / y_std
y_test = (y_test - y_mean) / y_std

# Divide the data into 75% training and 25% validation data sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, shuffle=True, random_state=1)

# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Val  : X=%s, y=%s' % (X_val.shape, y_val.shape))
print('Test : X=%s, y=%s' % (X_test.shape, y_test.shape))

Train: X=(237, 2), y=(237, 4)
Val  : X=(80, 2), y=(80, 4)
Test : X=(80, 2), y=(80, 4)


In [12]:
# Initialize neural network
NN = Sequential()

NN.add(Dense(50, input_shape=(2, )))
NN.add(Activation('relu'))

NN.add(Dense(25))
NN.add(Activation('relu'))

NN.add(Dense(10))
NN.add(Activation('relu'))

NN.add(Dense(4))

# Compile and train the neural network
lr = 0.01
mom = 0.9 
ep = 20
bs = 100

opt = SGD(learning_rate=lr, momentum=mom)
NN.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
history = NN.fit(X_train, y_train, epochs=ep, batch_size=bs, validation_data=(X_val, y_val), verbose=1)

# Predict and calculate loss
y_predict = NN.predict(X_test)
error = y_test - y_predict
SSE = np.sum(error ** 2)
print(SSE)

Epoch 1/20
3/3 [==============================] - 1s 144ms/step - loss: 1.0885 - mean_squared_error: 1.0885 - val_loss: 0.9739 - val_mean_squared_error: 0.9739
Epoch 2/20
3/3 [==============================] - 0s 26ms/step - loss: 1.0347 - mean_squared_error: 1.0347 - val_loss: 0.9061 - val_mean_squared_error: 0.9061
Epoch 3/20
3/3 [==============================] - 0s 60ms/step - loss: 0.9636 - mean_squared_error: 0.9636 - val_loss: 0.8670 - val_mean_squared_error: 0.8670
Epoch 4/20
3/3 [==============================] - 0s 44ms/step - loss: 0.9187 - mean_squared_error: 0.9187 - val_loss: 0.8271 - val_mean_squared_error: 0.8271
Epoch 5/20
3/3 [==============================] - 0s 25ms/step - loss: 0.8713 - mean_squared_error: 0.8713 - val_loss: 0.7872 - val_mean_squared_error: 0.7872
Epoch 6/20
3/3 [==============================] - 0s 26ms/step - loss: 0.8226 - mean_squared_error: 0.8226 - val_loss: 0.7470 - val_mean_squared_error: 0.7470
Epoch 7/20
3/3 [=============================